In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
from torch.utils.data import DataLoader
from tqdm import tqdm

# Define the Model with SEBlock and Enhanced Architecture
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.fc1 = nn.Linear(channels, channels // reduction)
        self.fc2 = nn.Linear(channels // reduction, channels)

    def forward(self, x):
        b, c, _, _ = x.size()
        y = x.view(b, c, -1).mean(2)  # Global Average Pooling
        y = torch.relu(self.fc1(y))
        y = torch.sigmoid(self.fc2(y))
        y = y.view(b, c, 1, 1)
        return x * y

class CIFAR10EnhancedModel(nn.Module):
    def __init__(self, num_classes=10):
        super(CIFAR10EnhancedModel, self).__init__()
        def dw_sep_conv(in_ch, out_ch, stride=1):
            return nn.Sequential(
                nn.Conv2d(in_ch, in_ch, 3, stride, 1, groups=in_ch, bias=False),
                nn.BatchNorm2d(in_ch),
                nn.GELU(),
                nn.Conv2d(in_ch, out_ch, 1, bias=False),
                nn.BatchNorm2d(out_ch),
                nn.GELU(),
                SEBlock(out_ch)
            )

        def inverted_residual(in_ch, out_ch, expansion, stride):
            mid_ch = in_ch * expansion
            return nn.Sequential(
                nn.Conv2d(in_ch, mid_ch, 1, bias=False),
                nn.BatchNorm2d(mid_ch),
                nn.GELU(),
                dw_sep_conv(mid_ch, out_ch, stride),
            ) if stride > 1 or in_ch != out_ch else nn.Sequential(
                nn.Conv2d(in_ch, mid_ch, 1, bias=False),
                nn.BatchNorm2d(mid_ch),
                nn.GELU(),
                dw_sep_conv(mid_ch, out_ch, stride),
            )

        self.stem = nn.Sequential(
            nn.Conv2d(3, 32, 3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.GELU(),
        )

        self.features = nn.Sequential(
            dw_sep_conv(32, 64, 1),
            inverted_residual(64, 128, 4, 2),
            inverted_residual(128, 128, 4, 1),
            inverted_residual(128, 256, 4, 2),
            dw_sep_conv(256, 512, 2),
        )

        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.stem(x)
        x = self.features(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [ ]:
!sudo apt update
!sudo apt install nvidia-driver-470

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,224 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,619 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Pac

In [2]:
#when using cpu and no new changes
import torch
import torchvision
import torchvision.transforms as transforms
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
from torch.utils.data import DataLoader
from tqdm import tqdm

def train_model():
    # Data Augmentation and Preprocessing
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

    # Model, Loss, Optimizer, Scheduler
    model = CIFAR10EnhancedModel().to('cpu')
    criterion = nn.CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=3e-4)
    scheduler = CosineAnnealingLR(optimizer, T_max=50)

    # Training Loop
    for epoch in range(50):
        model.train()
        running_loss = 0.0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
        for inputs, targets in pbar:
            inputs, targets = inputs.to('cpu'), targets.to('cpu')

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(loss=running_loss / len(train_loader))

        scheduler.step()

        # Validation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to('cpu'), targets.to('cpu')
                outputs = model(inputs)
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()

        acc = 100.0 * correct / total
        print(f"Validation Accuracy after Epoch {epoch+1}: {acc:.2f}%")

    # Save Model
    torch.save(model.state_dict(), "cifar10_light_model.pth")
    print("Model training complete. Saved as 'cifar10_light_model.pth'")

if __name__ == "__main__":
    train_model()

100%|██████████| 170M/170M [00:13<00:00, 12.6MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


NameError: name 'CIFAR10LightModel' is not defined

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.optim import SGD
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader
from tqdm import tqdm

def train_model():
    # Data Augmentation and Preprocessing
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    # Load CIFAR-10 Dataset
    train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

    # Model, Loss, Optimizer, Scheduler
    model = CIFAR10EnhancedModel().to('cuda')
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
    scheduler = OneCycleLR(optimizer, max_lr=0.1, steps_per_epoch=len(train_loader), epochs=50)

    # Training Loop
    for epoch in range(50):
        model.train()
        running_loss = 0.0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
        for inputs, targets in pbar:
            inputs, targets = inputs.to('cuda'), targets.to('cuda')

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(loss=running_loss / len(train_loader))

        scheduler.step()

        # Validation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to('cuda'), targets.to('cuda')
                outputs = model(inputs)
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()

        acc = 100.0 * correct / total
        print(f"Validation Accuracy after Epoch {epoch+1}: {acc:.2f}%")

    # Save Model
    torch.save(model.state_dict(), "cifar10_enhanced_model.pth")
    print("Model training complete. Saved as 'cifar10_enhanced_model.pth'")

if __name__ == "__main__":
    train_model()

Files already downloaded and verified
Files already downloaded and verified


Epoch 1: 100%|██████████| 391/391 [01:15<00:00,  5.18it/s, loss=1.63]


Validation Accuracy after Epoch 1: 60.80%


Epoch 2: 100%|██████████| 391/391 [01:08<00:00,  5.74it/s, loss=1.31]


Validation Accuracy after Epoch 2: 68.70%


...

Validation Accuracy after Epoch 48: 90.19%


Epoch 49: 100%|██████████| 391/391 [01:07<00:00,  5.79it/s, loss=0.668]


Validation Accuracy after Epoch 49: 90.61%


Epoch 50: 100%|██████████| 391/391 [01:07<00:00,  5.82it/s, loss=0.661]


Validation Accuracy after Epoch 50: 89.63%
Model training complete. Saved as 'cifar10_enhanced_model.pth'


In [6]:
from fvcore.nn import FlopCountAnalysis
model = CIFAR10EnhancedModel()
inputs = torch.randn(1, 3, 32, 32)
flops = FlopCountAnalysis(model, inputs)
print(f"FLOPs: {flops.total() / 1e6:.2f}M")

FLOPs: 96.31M


In [7]:
torch.save(model.state_dict(), "cifar10_model.pth")
import os
print(f"Model Size: {os.path.getsize('cifar10_model.pth') / 1e6:.2f} MB")


Model Size: 2.41 MB


In [9]:
model =  CIFAR10EnhancedModel() # Initialize your model

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total Trainable Parameters: {total_params}")


Total Trainable Parameters: 584718


In [10]:
pip install torchsummary


In [5]:
!pip install fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61396 sha256=9da74df784250ce0be4283434fdbba55cdd1939acca4c4fb820bcb864fbb0383
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31528 sha256=e2b0dac4f1cb902f35574ae6aba6f1551b2e5d235f13ae37b8b6ab9e4f1c8f20
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath
